In [1]:
from sessions import *
from venueordering import *
from pylatex_textboxes import *
from formatting_solver import *

In [2]:
import itertools as it
import os
import requests
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import BackendApplicationClient

def connect_api_sess():
    client_secret = os.environ["GREN_CLIENT_SECRET"]
    assert len(client_secret)>0
    client_id = os.environ["GREN_CLIENT_ID"]
    base_url = os.environ["GREN_BASE_URL"]
    event="swancon2017"

    client = BackendApplicationClient(client_id=client_id)
    oath_sess = OAuth2Session(client=client)
    token = oath_sess.fetch_token(token_url=base_url+'oauth/token', client_id=client_id, client_secret=client_secret)
    return oath_sess, base_url, event


In [3]:
def all_venues(sessions):
    return set().union(*(ss.venues for ss in sessions))

In [4]:
def load_grenadine_session(prog_item):
    title = prog_item['title']
    print(title)
    id = prog_item['id']
    start = prog_item['start_time']
    end = prog_item['end_time']
    
    people = [entry['person']['full_publication_name'] for entry in prog_item['sorted_published_item_assignments']]
    
    tags = [entry['name'] for entry in prog_item['base_tags']]
    try:
        tags.append(prog_item['format']['name'])
    except KeyError:
        pass
    if prog_item['is_break']:
        tags.append('Break')
        
    
    try:
        venues = [prog_item['published_room']['name']]
    except KeyError:
        venues=[]
        pass
    
        
    description = prog_item['description']
    return session(id, start, end, title, tags, people, venues, description)



def fetch_sessions_raw():
    api_sess, base_url, event = connect_api_sess() 
    url = base_url+event+"/planner_apis/1/published_items" 
    return api_sess.get(url, params={'paginated':'false'})
    
    
def fetch_sessions():
    result = fetch_sessions_raw()
    raw_sessions = result.json()['response']
    return list(map(load_grenadine_session, raw_sessions))


In [5]:
srs = fetch_sessions_raw()

In [6]:
srs.json()['response']

[{'base_tags': [],
  'card_size': 1,
  'child_ids': [],
  'created_at': '2017-03-04T10:09:56+08:00',
  'description': '',
  'duration': 90,
  'end_time': '2017-04-13T18:30:00+08:00',
  'id': 1102,
  'images': [],
  'is_break': True,
  'parent_id': None,
  'requires_signup': False,
  'sorted_published_item_assignments': [],
  'start_time': '2017-04-13T17:00:00+08:00',
  'theme_names': [],
  'ticket_type_ids': [],
  'title': 'Dinner',
  'updated_at': '2017-04-06T18:55:14+08:00'},
 {'base_tags': [],
  'card_size': 1,
  'child_ids': [],
  'created_at': '2017-04-01T08:38:19+08:00',
  'description': '',
  'duration': 90,
  'end_time': '2017-04-13T18:30:00+08:00',
  'format': {'id': 35, 'images': [], 'name': 'Closed'},
  'id': 1352,
  'images': [],
  'is_break': False,
  'parent_id': None,
  'published_room': {'id': 47,
   'name': 'Panorama South Rear  (Art Show)',
   'published_venue_id': 2},
  'requires_signup': False,
  'sorted_published_item_assignments': [{'person': {'company': '',
     

In [7]:
def replace_room_with_rooms(sessions, old, *news):
    for sess in sessions:
        if old in sess.venues:
            #print(sess.title)
            sess.venues.remove(old)
            sess.venues.update(news)
    
    sessions
    

"Find multiple copies of events for different rooms at the same time, and convert them to single events in spanning rooms"
def bridge_rooms(sessions):
    bridged_sessions = []
    
    sessions.sort(key=lambda x: (x.start, x.end, x.title))
    for _, matchs in it.groupby(sessions, lambda x: (x.start, x.end, x.title)):
        matchs = list(matchs) #GoldPlate: this doesn't have to be done
        head = matchs[0]
        tail = matchs[1:]
        for sess in tail:
            head.venues.update(sess.venues)
        bridged_sessions.append(head)
        
    
    sessions[:]=bridged_sessions[:]
    return sessions

def fill_empty_room_slots(sessions):
    venues = all_venues(sessions)
    for ss in sessions:
        if len(ss.venues)==0:
            print(ss.title)
            ss.venues = venues
    sessions


In [9]:
sessions = fetch_sessions()
replace_room_with_rooms(sessions, 'Swan Rooms', 'Swan Room, Black', 'Swan Room, White')
bridge_rooms(sessions);
fill_empty_room_slots(sessions)

Dinner
Art Show Registration & Set Up
Hitchhiker's Guide to Swancon (Opening Ceremony)
Welcome To Swancon
Quilting with Davina
Board Gaming
Meet The AV Crew
Art Show Registration & Set Up
Open Console Gaming
Safe Spaces
Patrick Troughton Remembered
The Fantastic History of Wearable Art
Immortal Futures: Living On A Post-death Planet
Room Closed
Geek Sing-along: The Return of the Sing
Douglas Adams: A Life Remembered
Hanna- Barbara: The Final Years
Board Gaming
Art Show Registration & Set Up
Open Console Gaming
42 Years of Swancon
Reading and Q & A Session with Wesley Chu
The Regeneration Game
Traders
Origami with Sanny Ang
Art Show
Run Through Banner
Forza Horizon 3 Time Trials
Walt Disney: The Second Renaissance
Modern Medieval in Perth
Xenobiology
Agnostic Photographer: Learning Your Camera
Frankenplushies
Buffy: An Appreciation
Best & Worst Reads of 2016
Agnostic Photographer: Practical
Plush Toy Wars
Lunch
Cloning
Girls und Panzer: Der Panel
Joyce Chng/Damask GOH Speech
Role Playin

In [10]:
set(tuple(cc.venues) for cc in sessions)

{('Board Room',),
 ('City Lights',),
 ('Cygnet Room',),
 ('Hovia Room',),
 ('Panorama North',),
 ('Panorama South',),
 ('Panorama South Rear  (Art Show)',),
 ('Swan Room, Black',),
 ('Swan Room, White',),
 ('Swan Room, White', 'Swan Room, Black'),
 ('Swan Room, White',
  'Swan Room, Black',
  'Panorama North',
  'Cygnet Room',
  'Panorama South Rear  (Art Show)',
  'Hovia Room',
  'City Lights',
  'Panorama South',
  'Board Room')}

In [ ]:
    api_sess, base_url, event = connect_api_sess() 
    url = base_url+event+"/planner_apis/1/published_rooms" 
    result =  api_sess.get(url, params={'paginated':'false'})
    raw_rooms = result.json()['response']
    fetched_venues = [(vv['sort_order'], vv['name']) for vv in raw_rooms]
    fetched_venues

In [11]:
def fetch_venue_orderer():
    api_sess, base_url, event = connect_api_sess() 
    url = base_url+event+"/planner_apis/1/published_rooms" 
    result =  api_sess.get(url, params={'paginated':'false'})
    raw_rooms = result.json()['response']
    fetched_venues = [vv['name'] for vv in raw_rooms]
    
    def orderer(sessions):
        used_venues = set.union(*(set(sess.venues) for sess in sessions))
        return [vn for vn in fetched_venues if vn in used_venues]
        
    return orderer

In [12]:
fetch_venue_orderer()(sessions)

['Swan Room, Black',
 'Swan Room, White',
 'City Lights',
 'Cygnet Room',
 'Panorama North',
 'Panorama South',
 'Panorama South Rear  (Art Show)',
 'Hovia Room',
 'Board Room']

In [13]:
def get_tag_colors_breaks_only(sessions):
    return {'Break': 'red!10!white'}

In [14]:
from pylatex.utils import escape_latex, NoEscape
from pylatex.utils import NoEscape
from pagelayout import Multicols
from itertools import groupby
from pylatex.base_classes import Environment

class Minipage(Environment):
    def __init__(self, width):
        Environment.__init__(self,arguments=[width])


def write_descriptions(sessions, doc, preprocess_text=lambda x:x):
    for day_name, day_session in groupby(sessions, lambda ss: ss.day):
        #with doc.create(Section(day_name,numbering=False)):
            with doc.create(Multicols(2)):
                doc.append(NoEscape("[\section*{%s}]" %day_name))
                for session in day_session:
                    if len(session.description)==0:
                        continue
                    title = session.title
                    if "AdultsOnly" in session.tags:
                        title+=NoEscape(" [ADULT]")
                    with doc.create(Subsection(title,numbering=False)):
                        with doc.create(Description()) as desc:
                            doc.append(Command("setlength",[NoEscape("\itemsep"),"0pt"]))
                            doc.append(Command("setlength",[NoEscape("\parsep"),"0pt"]))
                            doc.append(Command("setlength",[NoEscape("\parskip"),"0pt"]))
                            desc.add_item("When:", session.day+", "+session.start_time+" -- "+session.end_time)

                            if len(session.venues)>0:
                                desc.add_item("Where:", ", ".join(session.venues))
                            if len(session.people)>0:
                                desc.add_item("Who:", ", ".join(session.people))
                            #if len(session.tags)>0:
                            #    desc.add_item("Tags:", ", ".join(session.tags))
                        doc.append(preprocess_text(session.description))
                doc.append(Command("newpage"))

In [15]:
import pylatex
from pylatex import Document, Section, Subsection, Subsubsection, Table, Package, lists
from pylatex.lists import Description

from pylatex.utils import escape_latex, NoEscape
from itertools import groupby
from pylatex.base_classes.command import Options
from pylatex.utils import escape_latex

def write_venues(doc, tt_solver):
    doc.append(textpos_origin('0.9cm','1cm'))
    doc.append(TextcolorboxStyle('sharp corners','center upper', valign='center',
                                 colframe='blue!50!black',colback='blue!10!white',
                                 boxsep='0pt',top='0mm',bottom='0mm',left='0mm',right='1mm'))
            
    for venue in tt_solver.venues:
        
        venue_words = venue.split()
        if len(venue_words) == 2:
            #Split the string onto two lines if it exactly 2 words
            venue_text = venue_words[0] + '\n' + venue_words[1]
        else:
            venue_text = venue
        
        doc.append(FixedTextbox(venue_text,
                                tt_solver.get_venue_x(venue),
                                '0cm', 
                                '1.5cm',
                                tt_solver.get_venue_width())
                               )

def make_pretty_timetable(doc,sessions, tt_solver):
        
    for date, day_sessions in groupby(sessions, lambda ss: ss.start.date()):   
        day_str = str(DAYS[date.weekday()])
        with doc.create(Subsection(NoEscape(day_str+" \hfill "+day_str+" \hfill "+day_str), numbering=False)):
           
            write_venues(doc,tt_solver)
            doc.append(TextcolorboxStyle('rounded corners', 'center upper', valign='center',
                                 colframe='blue!50!black',colback='white!10!white',
                                 boxsep='1pt',top='0mm',bottom='0mm',left='0mm',right='0mm'))
            
            day_sessions = sorted(day_sessions, key = lambda ss: -len(ss.venues) or -len(tt_solver.venues)-1)
            for session in day_sessions:
                #print("*", session.title)
                colback = tt_solver.get_color(session)
                tcb_options = Options(colback=colback) if colback else None
                doc.append(FixedTextbox(NoEscape('%s \\\\ \\tcbfontsize{0.75} %s -- %s ' % 
                                                     tuple(map(escape_latex, (session.title, session.start_time, session.end_time)))),
                                        tt_solver.get_x(session),
                                        tt_solver.get_y(session), 
                                        tt_solver.get_height(session),
                                        tt_solver.get_width(session),
                                        tcb_options=tcb_options))

            doc.append(Command('newpage'))

In [16]:
doc = Document(documentclass="scrreprt")
margins=['tmargin=0.5cm','bmargin=1.5cm','lmargin=1.5cm','rmargin=1cm',]
doc.packages.append(Package('geometry', options=margins))
doc.packages.append(Package('xcolor', options=["svgnames","dvipsnames"]))
doc.packages.append(Package("microtype"))

#doc.packages.append(Package("draftwatermark"))
doc.append(Command("newgeometry",arguments=",".join(margins)))
#doc.append(Command("SetWatermarkText", "Draft v0.7.0"))
#doc.append(Command("SetWatermarkScale", "0.5"))
#doc.append(Command("SetWatermarkColor", "0.9,0.3,0.3", "rgb"))


#############
tt_solver = timetable_metric_solver(sessions,
                                    hour_len=1.7,
                                    venue_width=2.15,
                                    units='cm', 
                                    overlap=0.05,
                                    voffset=1.7,             
                                    venue_orderer = fetch_venue_orderer(),
                                    get_tag_colors = get_tag_colors_breaks_only
                                    )

make_pretty_timetable(doc,sessions, tt_solver)
write_descriptions(sessions,doc, resolve_html)

###############
with open("out/exported.tex", 'w') as fh:
    doc.dump(fh)
#####
    
from IPython.display import FileLink, FileLinks
    
!lualatex --output-directory=out --interaction=nonstopmode out/exported.tex
FileLinks("./out")


This is LuaTeX, Version beta-0.79.1 (TeX Live 2015/dev/Debian) (rev 4971) 
 restricted \write18 enabled.
(./out/exported.tex
LaTeX2e <2014/05/01>
Babel <3.9l> and hyphenation patterns for 79 languages loaded.
(/usr/share/texlive/texmf-dist/tex/latex/koma-script/scrreprt.cls
Document Class: scrreprt 2013/12/19 v3.12 KOMA-Script document class (report)
(/usr/share/texlive/texmf-dist/tex/latex/koma-script/scrkbase.sty
(/usr/share/texlive/texmf-dist/tex/latex/koma-script/scrbase.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/latex/koma-script/scrlfile.sty
Package scrlfile, 2013/12/19 v3.12 KOMA-Script package (loading files)
                  Copyright (C) Markus Kohm

))) (/usr/share/texlive/texmf-dist/tex/latex/koma-script/tocbasic.sty)
(/usr/share/texlive/texmf-dist/tex/latex/koma-script/scrsize11pt.clo)
(/usr/share/texlive/texmf-dist/tex/latex/koma-script/typearea.sty
Package typearea, 2013/12/19 v3.12 KOMA-Script package (type area

./out/
  exported.pdf
  exported_days.pdf
  exported.log
  exported_days.aux
  exported_days.tex
  exported_days.log
  exported.tex
  exported.aux